# EMOE

## Descarga de los Datos

### Lectura de los datos.

In [68]:
import pandas as pd
import numpy as np

emoe = pd.read_excel('data\\fd_emoe_serie2018.xlsx', sheet_name=None, skiprows= 1)
ek = list(emoe.keys())
ek

['CONSTRUCCION', 'IND MANUFACTURERAS', 'COMERCIO', 'SERVICIOS']

Como podemos observar, tenemos 4 hojas que debemos formatear. 

## Análisis de Datos.

In [69]:
sheet = ek[0]
data = emoe[sheet]
df = data.copy()
df.head()

,Variable,Descripción,Observaciones
0,1,OBRAS EJECUTADAS,NaN
1,p1_1est,Obras Ejecutadas,Es el valor total o parcial (avance de obra) d...
2,p1_1estpo,Porcentaje de variación en Obras ejecutadas,NaN
3,NaN,OBRAS EJECUTADAS COMO SUBCONTRATISTA,NaN
4,p1_2est,Obras ejecutas como subcontratista,Es el valor total o parcial (avance de obra) d...


Observando todo la hoja, vemos que algunas columnas poseen números. Las usaremos para subdividir la hoja en pequeñas tablas.

In [70]:
var_num = list(df.iloc[:, 0].agg(type) == int)
var_num

C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\3659518525.py:1: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)


[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

Una vez los detectamos. Mostramos únicamente las columnas que posean un valor de verdadero. 

In [71]:
i = 0
first_index = []
for value in var_num:
    if value:
        first_index.append(i)
    i += 1
first_index

[0, 11, 20, 30, 39, 42]

Una vez los tenemos localizados, procedemos a crear las distintas subtablas. 

In [72]:
first_level = []
for i in range(len(first_index) - 1):
    first_level.append(df.iloc[first_index[i]: first_index[i + 1],:])
first_level.append(df.iloc[first_index[i + 1]: ,:])

Ahora seleccionamos una de las subtablas para formatear. 

In [73]:
first_level[5]
subdata = first_level[5]
subdf = subdata.copy()
subdf

,Variable,Descripción,Observaciones
42,7,SITUACIÓN ECONOMICA DEL PAÍS,NaN
43,s8_1,Situación económica del país actual,Las condiciones económicas prevalecientes en e...
44,s8_2,Situación económica del país futura,NaN
45,NaN,SITUACIÓN ECONOMICA DE LA EMPRESA,NaN
46,s8_3,Situación económica de la empresa actual,Las condiciones económicas prevalecientes en l...
47,s8_4,Situación económica de la empresa futura,NaN


Ahora procederemos a realizar algunos pasos para moldear los dataframes. 

In [74]:
# Agregamos un NA donde se encontraba el número con tal de homogeniezar la información
subdf.iloc[0, 0] = pd.NA
subdf['nna'] = subdf.isna().sum(axis = 1)
# Reseteamos los indices por estabilidad. 
subdf = subdf.reset_index(drop=True)  

# Capturamos los indices de los nombres de las subsubtablas. 
seltwo = subdf[subdf['nna'] == 2]
selindex = list(seltwo.index)

# Hacemos la subsubdivisión.
second_level = []
for i in range(len(selindex) - 1):
    second_level.append(subdf.iloc[selindex[i]:selindex[i + 1], :])
second_level.append(subdf.iloc[selindex[i + 1]:, :])

# Seleccionamos una de las subsubtablas
subsubdata = second_level[0]
subsubdf = subsubdata.copy()

subsubname = subsubdf.iloc[0, 1] # Nombre de la Tabla.
subsubdf.iloc[:, -1] = subsubname
subsubdf = subsubdf.iloc[1:, :] # Removemos la primer columna. 

subsubdf # Mostramos el resultado. 

,Variable,Descripción,Observaciones,nna
1,s8_1,Situación económica del país actual,Las condiciones económicas prevalecientes en e...,SITUACIÓN ECONOMICA DEL PAÍS
2,s8_2,Situación económica del país futura,NaN,SITUACIÓN ECONOMICA DEL PAÍS


Ahora, agrupamos todo para cada elemento de second_level. 

In [75]:
subsublist = second_level
for i, subsub in enumerate(subsublist):
    subsubdata = subsub
    subsubdf = subsubdata.copy()
    subsubname = subsubdf.iloc[0, 1] # Nombre de la Tabla.
    subsubdf.iloc[:, -1] = subsubname
    subsubdf = subsubdf.iloc[1:,:] # Removemos la primer columna.
    if i == 0:
        ssdf = subsubdf.copy()
    else:
        ssdf = pd.concat([ssdf, subsubdf])


## El mismo codigo, pero ahora como función. 
def last_level(subsublist: list):
    for i, subsub in enumerate(subsublist):
        subsubdf = subsub.copy()
        subsubname = subsubdf.iloc[0, 1] # Nombre de la Tabla.
        subsubdf.iloc[:, -1] = subsubname
        subsubdf = subsubdf.iloc[1:,:] # Removemos la primer columna.
        if i == 0:
            ssdf = subsubdf.copy()
        else:
            ssdf = pd.concat([ssdf, subsubdf])
    return ssdf 

ssdf

,Variable,Descripción,Observaciones,nna
1,s8_1,Situación económica del país actual,Las condiciones económicas prevalecientes en e...,SITUACIÓN ECONOMICA DEL PAÍS
2,s8_2,Situación económica del país futura,NaN,SITUACIÓN ECONOMICA DEL PAÍS
4,s8_3,Situación económica de la empresa actual,Las condiciones económicas prevalecientes en l...,SITUACIÓN ECONOMICA DE LA EMPRESA
5,s8_4,Situación económica de la empresa futura,NaN,SITUACIÓN ECONOMICA DE LA EMPRESA


Una vez tenemos el código, iremos dando pasos hacia atrás. 

In [76]:
def back_level(sublist: list):
    list_levels = []
    for i, sub in enumerate(sublist):
        subdf = sub.copy()
        subdf.iloc[0, 0] = pd.NA
        subdf['nna'] = subdf.isna().sum(axis = 1)
        subdf = subdf.reset_index(drop=True)  
        seltwo = subdf[subdf['nna'] == 2]
        selindex = list(seltwo.index)
        second_level = []
        for i in range(len(selindex) - 1):
            second_level.append(subdf.iloc[selindex[i] : selindex[i + 1], :])
        second_level.append(subdf.iloc[selindex[-1]: ,:])
        list_levels.append(second_level)
    return list_levels

Finalmente. Juntando todo, tenemos la primera hoja.

In [77]:
sheet = ek[0]
data = emoe[sheet]
df = data.copy()

def data_sheet(df: pd.DataFrame):
    var_num = list(df.iloc[:, 0].agg(type) == int)
    i = 0
    first_index = []
    for value in var_num:
        if value:
            first_index.append(i)
        i += 1
    for i in range(len(first_index) - 1):
        first_level.append(df.iloc[first_index[i]: first_index[i + 1],:])
    first_level.append(df.iloc[first_index[-1]: ,:])
    levels = back_level(first_level)
    for k, level in enumerate(levels):
        if k == 0:
            sdf = last_level(level)
        else:
            sdf = pd.concat([sdf, last_level(level)])
    return sdf
one_sheet = data_sheet(df)


C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)


In [78]:
test = one_sheet.reset_index(drop=True)
test.to_excel('results\\test.xlsx')

Ahora queda revisar que sucede con el resto de hojas, primero una ejecución confiada. 

In [79]:
sheets = list(emoe.keys())

for s, sh in enumerate(sheets):
    data = emoe[sh]
    df = data.copy()
    if s == 0:
        last_df = data_sheet(df)
    else:
        last_df = pd.concat([last_df, data_sheet(df)])

C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)
C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)
C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)
C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.i

Todo se ejecuto sin problemas, entonces generamos el excel. 

In [80]:
last_df.to_excel('results\\emoe2018.xlsx')

El archivo ya esta terminado, sin embargo vamos a darle los toques finales. 

In [81]:
sheets = list(emoe.keys())

for s, sh in enumerate(sheets):
    data = emoe[sh]
    df = data.copy()
    if s == 0:
        last_df = data_sheet(df)
        last_df = last_df.drop_duplicates(ignore_index=True)
    else:
        f_df = data_sheet(df)
        last_df = pd.concat([last_df, f_df])
        last_df = last_df.drop_duplicates(ignore_index=True)
    
last_df.rename({'nna':'SubNombre'})
last_df.to_excel('results\\emoe2018.xlsx')

C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)
C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)


C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)
C:\Users\Tamara\AppData\Local\Temp\ipykernel_6276\2170344279.py:6: FutureWarning: using <class 'type'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  var_num = list(df.iloc[:, 0].agg(type) == int)
